<a href="https://colab.research.google.com/github/talfig/Sign-Language/blob/main/Sign_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [3]:
train = pd.read_csv("/content/drive/MyDrive/Sign_Language/sign_mnist_train.csv")
test = pd.read_csv("/content/drive/MyDrive/Sign_Language/sign_mnist_test.csv")

In [4]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [5]:
test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53


In [27]:
class AlexNet(nn.Module):
  def __init__(self, dropout=0.5):
    super().__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 64, 11, stride=4, padding=2),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3, stride=2),
      nn.Conv2d(64, 192, 5, padding=2),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3, stride=2),
      nn.Conv2d(192, 384, 3, padding=1),
      nn.ReLU(),
      nn.Conv2d(384, 256, 3, padding=1),
      nn.ReLU(),
      nn.Conv2d(256, 256, 3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3, stride=2)
      )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.classifier = nn.Sequential(
      nn.Dropout(p=dropout),
      nn.Linear(256 * 6 * 6, 4096),
      nn.ReLU(),
      nn.Dropout(p=dropout),
      nn.Linear(4096, 4096),
      nn.ReLU(),
      nn.Linear(4096, 24),
      nn.Softmax(dim=1)
      ).to(device)

  def forward(self, x):
    x = self.features(x)
    x = self.classifier(x.flatten(start_dim=1))
    return x

model = AlexNet(0.45)
epochs = 750 # number of single passes on the network
loss_fnPrecents = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [36]:
train_dataloader = DataLoader(train, batch_size=100)

In [28]:
def train_model(model, dataloader, epochs, optimizer, loss_fn):
    for epoch in range(epochs):
        model.train() # Set the model to training mode
        for data, targets in dataloader:
            data, targets = data.to(model.device), targets.to(model.device).long().squeeze()
            optimizer.zero_grad()
            outputs = model(data)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()

def evaluate_model(model, dataloader):
    model.eval() # Set the model to evaluation mode
    total, correct = 0, 0
    with torch.no_grad():
        for data, targets in dataloader:
            data, targets = data.to(model.device), targets.to(model.device).long().squeeze()
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return 100 * correct / total